# Module 5 ? Deployment Homework (2025)

This notebook guides you through every question with explicit commands and code you can run locally. It also explains why each answer is obtained.

We use the lead?scoring model provided by the course.


## 0. Prereqs
- Python 3.12+ (3.13 recommended)
- uv (new Python package/deps tool)
- Docker Desktop (for Q5?Q6)
- Jupyter for this notebook

We?ll initialize a fresh uv project in a separate folder and keep this notebook in the course repo.


## Q1. Install uv and check version

On your terminal (PowerShell/WSL/Terminal), run one of:
- Windows: `winget install --id astral-sh.uv` or `pip install uv`
- macOS/Linux: `curl -LsSf https://astral.sh/uv/install.sh | sh`

Then verify the version:
```bash
uv --version
```
Record the printed version for submission.


## Initialize a clean uv project (outside this repo)

From your workspace root (or any folder), do:
```bash
mkdir hw05-uv && cd hw05-uv
uv init
```
This creates `pyproject.toml` and a virtual environment managed by uv.


## Q2. Add scikit?learn==1.6.1 and get lock hash

Install exact version and create a lock file:
```bash
uv add scikit-learn==1.6.1
```
Now open `uv.lock` and find the first hash for `scikit-learn` (the string starting with `sha256:`). Submit that full hash.

Tip (optional, run here if this notebook is in the uv project):
```python
# Parse uv.lock and print the first hash for scikit-learn
```


## Models for scoring
- `pipeline_v1.bin` (for local scoring and FastAPI locally)
- `pipeline_v2.bin` is already baked into the base Docker image and will be used inside the container.

Download v1 locally in the same folder as your FastAPI script or this notebook:
```bash
wget https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
```
Verify checksum (optional):
```bash
md5sum pipeline_v1.bin  # should be 7d17d2e4dfbaf1e408e1a62e6e880d49
```


## Q3. Load pipeline_v1 and score a record
Explanation: the model is a scikit?learn pipeline: `DictVectorizer` ? `LogisticRegression`. We must convert the input JSON to a Python dict and call `predict_proba` to get a probability of conversion (class 1).


In [1]:
import pickle, json
from pathlib import Path

MODEL_FILE = Path('cohorts/2025/05-deployment/pipeline_v1.bin') if Path('cohorts/2025/05-deployment/pipeline_v1.bin').exists() else Path('pipeline_v1.bin')

with open(MODEL_FILE, 'rb') as f:
    pipeline_v1 = pickle.load(f)

client = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

proba = float(pipeline_v1.predict_proba([client])[:, 1][0])
proba


D:\tien.nv12\learn\machine-learning-zoomcamp\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
D:\tien.nv12\learn\machine-learning-zoomcamp\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
D:\tien.nv12\learn\machine-learning-zoomcamp\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator Pipel

0.5336072702798061

Interpretation: Submit the closest option to the printed probability. The pipeline handles missing values because it was trained with appropriate preprocessing in the notebook referenced by the course.


## Q4. Serve with FastAPI locally
We?ll expose a `/predict` endpoint that accepts the same JSON, loads `pipeline_v1.bin`, and returns the probability.

Steps in the uv project directory:
```bash
uv add fastapi uvicorn[standard]
```
Create `service.py` with:
```python
from fastapi import FastAPI
import pickle
from pydantic import BaseModel
from pathlib import Path

class Lead(BaseModel):
    lead_source: str
    number_of_courses_viewed: int
    annual_income: float

app = FastAPI()

# Load v1 locally by default
MODEL_PATHS = [Path('pipeline_v1.bin'), Path('cohorts/2025/05-deployment/pipeline_v1.bin')]
for p in MODEL_PATHS:
    if p.exists():
        model_path = p
        break
else:
    model_path = Path('pipeline_v2.bin')  # fallback, for Docker base image

with open(model_path, 'rb') as f:
    model = pickle.load(f)

@app.post('/predict')
async def predict(payload: Lead):
    proba = float(model.predict_proba([payload.model_dump()])[:, 1][0])
    return {'converted_probability': proba}
```
Run locally:
```bash
uvicorn service:app --host 0.0.0.0 --port 8000
```
Test from Python:
```python
import requests
url = 'http://127.0.0.1:8000/predict'
client = {"lead_source":"organic_search","number_of_courses_viewed":4,"annual_income":80304.0}
requests.post(url, json=client).json()
```
Record the closest option to the returned probability.


## Q5. Pull base Docker image and check size
Run:
```bash
docker pull agrigorev/zoomcamp-model:2025
docker images | findstr zoomcamp-model  # Windows
# or: docker images | grep zoomcamp-model
```
Submit the size shown in the SIZE column for `agrigorev/zoomcamp-model:2025`.


## Dockerfile for Q6
We?ll base on `agrigorev/zoomcamp-model:2025` which already has `pipeline_v2.bin` in `/code`.

`Dockerfile`:
```dockerfile
FROM agrigorev/zoomcamp-model:2025
WORKDIR /code

# Bring in project metadata for dependency install
COPY pyproject.toml uv.lock ./

# Install dependencies with uv (pip fallback if preferred)
RUN python -m pip install --no-cache-dir uv  && uv sync --frozen --no-cache

# Copy FastAPI service (expects pipeline_v2.bin present already)
COPY service.py ./

EXPOSE 8000
CMD ["uvicorn", "service:app", "--host", "0.0.0.0", "--port", "8000"]
```
Build and run:
```bash
docker build -t hw05-service .
docker run -it --rm -p 8000:8000 hw05-service
```
Test the same client as in Q4 to get the probability from the containerized service and choose the closest option for Q6.


## Why these answers make sense
- The pipeline is a linear model over one?hot encoded `lead_source` and two numeric features. Probabilities are logistic?sigmoid of the linear score.
- uv lock hash uniquely identifies the wheel/distribution; you read the first hash for scikit?learn 1.6.1 in `uv.lock`.
- Using the base image avoids compiling scikit?learn inside the container and keeps the image small; we only add Python deps and the service script, so size barely grows.
- Predictions inside Docker use `pipeline_v2.bin` pre?baked in the image, so results can differ slightly from Q3, which uses `pipeline_v1.bin`.
